# <center>Загрузка данных (daily) S&P500
Загрузка данных осуществляется с помощью сервиса [Alpha Vantage](https://www.alphavantage.co/). Здесь, с определенными ограничениями, можно бесплатно взять биржевые данные американских компаний.   

In [1]:
import os
os.chdir("..")

%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import requests
import pandas as pd
import time
import json
import shutil

In [3]:
import urllib
from bs4 import BeautifulSoup

In [4]:
path = 'prices_sp100_15min'

## Функция загрузки данных

In [5]:
APIKEY = os.getenv('APIKEY')

In [6]:
def load_data_15min(ticker, year, month, path=path, outputsize='full', apikey=APIKEY):
    filename = f'{ticker}_y{year}_m{month}.csv'
    path_to_file = os.path.join(path, filename)
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol={ticker}"\
          f"&outputsize={outputsize}&interval=15min&slice=year{year}month{month}&apikey={apikey}&datatype=csv"
    r = requests.get(url)
    content = r.content.decode('UTF-8')
    with open(path_to_file, "w") as file:
        file.write(content)
#     print(f'\tUpdate {ticker} price history:', path_to_file)
    return path_to_file

#### testing

In [40]:
# ticker = 'SPX'
ticker = 'SPX'
path_to_file = load_data_15min(ticker, 1, 2)

In [41]:
data = pd.read_csv(path_to_file)
data.head()

,time,open,high,low,close,volume


In [9]:
ticker = 'IBM'
path_to_file = load_data_15min(ticker, 1, 2)

In [10]:
data = pd.read_csv(path_to_file)

In [11]:
data.head()

,time,open,high,low,close,volume
0,2020-09-04 20:00:00,122.32,122.32,122.32,122.32,273
1,2020-09-04 19:45:00,122.40,122.40,122.40,122.40,165
2,2020-09-04 19:30:00,122.40,122.40,122.40,122.40,500
3,2020-09-04 18:45:00,122.40,122.44,122.40,122.40,750
4,2020-09-04 18:15:00,122.32,122.32,122.32,122.32,330


In [12]:
data.shape

(895, 6)

## Формируем список акций индекса S&P100

Разбор [страницы Википедии](https://en.wikipedia.org/wiki/S%26P_100) для получения списка акций, входящих в индекс S&P 100.    

In [12]:
url = "https://en.wikipedia.org/wiki/S%26P_100"
response = urllib.request.urlopen(url)
html = response.read()

In [13]:
soup = BeautifulSoup(html)

# Найдём таблицу:
table = soup.find('table', {'class': 'wikitable sortable'})

# Словарь для тикеров, сгруппированных по секторам экономики:
sp100_tickers = list()

for r in table.findAll('tr'):  # Цикл по строкам таблицы.
    col = r.findAll('td')      # Колонки таблицы.
    if len(col)>0:

        # В 1-й колонке таблицы - тикер акции:
        ticker = str(col[0].string.strip())
            
        # Добавляем тикер акции в список соответствующего сектора:
        sp100_tickers.append(ticker)

In [14]:
print(sp100_tickers)

['AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'AIG', 'ALL', 'AMGN', 'AMT', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BKNG', 'BLK', 'BMY', 'BRK.B', 'C', 'CAT', 'CHTR', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CRM', 'CSCO', 'CVS', 'CVX', 'DD', 'DHR', 'DIS', 'DOW', 'DUK', 'EMR', 'EXC', 'F', 'FB', 'FDX', 'GD', 'GE', 'GILD', 'GM', 'GOOG', 'GOOGL', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KHC', 'KMI', 'KO', 'LLY', 'LMT', 'LOW', 'MA', 'MCD', 'MDLZ', 'MDT', 'MET', 'MMM', 'MO', 'MRK', 'MS', 'MSFT', 'NEE', 'NFLX', 'NKE', 'NVDA', 'ORCL', 'OXY', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RTX', 'SBUX', 'SLB', 'SO', 'SPG', 'T', 'TGT', 'TMO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB', 'V', 'VZ', 'WBA', 'WFC', 'WMT', 'XOM']


In [15]:
len(sp100_tickers)

101

In [18]:
with open('sp100_tickers.json', 'w') as f:
    json.dump(sp100_tickers, f)

In [14]:
with open("sp100_tickers.json", "r") as f:
    sp100_tickers = json.load(f)
print(sp100_tickers)

['AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'AIG', 'ALL', 'AMGN', 'AMT', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BKNG', 'BLK', 'BMY', 'BRK.B', 'C', 'CAT', 'CHTR', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CRM', 'CSCO', 'CVS', 'CVX', 'DD', 'DHR', 'DIS', 'DOW', 'DUK', 'EMR', 'EXC', 'F', 'FB', 'FDX', 'GD', 'GE', 'GILD', 'GM', 'GOOG', 'GOOGL', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KHC', 'KMI', 'KO', 'LLY', 'LMT', 'LOW', 'MA', 'MCD', 'MDLZ', 'MDT', 'MET', 'MMM', 'MO', 'MRK', 'MS', 'MSFT', 'NEE', 'NFLX', 'NKE', 'NVDA', 'ORCL', 'OXY', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RTX', 'SBUX', 'SLB', 'SO', 'SPG', 'T', 'TGT', 'TMO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB', 'V', 'VZ', 'WBA', 'WFC', 'WMT', 'XOM']


## Скачиваем котировки акций согласно списку для индекса **S&P100**

Из-за некоторых ограничений на сервисе [Alpha Vantage](https://www.alphavantage.co/) данные скачиваем в несколько этапов и с периодической минутной задержкой

In [15]:
def download_one_position(ticker, num):
    for year in [1, 2]:
        for month in range(1, 13):
            path_to_file = load_data_15min(ticker, year, month, path='prices_sp100_15min_raw')
            num += 1
            if (num + 1) % 5 == 0:
                time.sleep(60)
    return num

In [ ]:
%%time

num = 0
for ticker in sp100_tickers:
    num = download_one_position(ticker, num)

In [16]:
sp100_tickers.index("PEP")

76

In [17]:
%%time

num = 0
for ticker in sp100_tickers[76:]:
    num = download_one_position(ticker, num)

Wall time: 2h 34min 39s


In [25]:
%%time

num = 0
for ticker in ['BRK-B']:
    num = download_one_position(ticker, num)

Wall time: 6min 36s


In [26]:
sp100_tickers_corr = [ticker.replace(".", "-") for ticker in sp100_tickers]
print(sp100_tickers_corr)

['AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'AIG', 'ALL', 'AMGN', 'AMT', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BKNG', 'BLK', 'BMY', 'BRK-B', 'C', 'CAT', 'CHTR', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CRM', 'CSCO', 'CVS', 'CVX', 'DD', 'DHR', 'DIS', 'DOW', 'DUK', 'EMR', 'EXC', 'F', 'FB', 'FDX', 'GD', 'GE', 'GILD', 'GM', 'GOOG', 'GOOGL', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KHC', 'KMI', 'KO', 'LLY', 'LMT', 'LOW', 'MA', 'MCD', 'MDLZ', 'MDT', 'MET', 'MMM', 'MO', 'MRK', 'MS', 'MSFT', 'NEE', 'NFLX', 'NKE', 'NVDA', 'ORCL', 'OXY', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RTX', 'SBUX', 'SLB', 'SO', 'SPG', 'T', 'TGT', 'TMO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB', 'V', 'VZ', 'WBA', 'WFC', 'WMT', 'XOM']


In [27]:
with open('sp100_tickers_corr.json', 'w') as f:
    json.dump(sp100_tickers_corr, f)

#### Примечание 
Индексы S&P500 (тикер SPX) и S&P100 (тикер OEX), 15-ти минутные, скачать нигде не удалось.   
В связи с этим прогнозировать будем курс **BRK-B** и, возможно, еще какую-то акцию.   
Выбор **BRK-B** связан с тем, что это фонд Баффета **BERKSHIRE HATHAWAY INC**, который, в свою очередь, сам состоит из акций и, в первом приближении, близок к индексам